In [1]:
import os
import pandas as pd
print(os.getcwd())

train_df = pd.read_csv('/home/lyq/DataSet/FakeNews/politifact/SC1_politifact/50_candidates_bm25_extended_reranking/Politifact.train.tsv',sep='\t')
dev_df = pd.read_csv('/home/lyq/DataSet/FakeNews/politifact/SC1_politifact/50_candidates_bm25_extended_reranking/Politifact.dev.tsv',sep='\t')
test_df = pd.read_csv('/home/lyq/DataSet/FakeNews/politifact/SC1_politifact/50_candidates_bm25_extended_reranking/Politifact.test.tsv',sep='\t')
test_hard_df = pd.read_csv('/home/lyq/DataSet/FakeNews/politifact/SC1_politifact/50_candidates_bm25_extended_reranking/Politifact.test2_hard.tsv',sep='\t')


print(f"train : {len(train_df['QueryID'].value_counts())}")
print(f"dev : {len(dev_df['QueryID'].value_counts())}")
print(f"test : {len(test_df['QueryID'].value_counts())}")
print(f"test_hard : {len(test_hard_df['QueryID'].value_counts())}")

df = pd.concat([train_df, dev_df, test_df, test_hard_df], ignore_index=True)

/home/lyq/DataSet/FakeNews/politifact
train : 1496
dev : 187
test : 187
test_hard : 156


In [4]:

df = df[['QueryID','QueryText','QueryImages','Label']]
df = df.drop_duplicates(keep='first',subset=['QueryID'])
df = df.dropna(subset=['QueryID','QueryText','QueryImages','Label']) 

values_count = df['Label'].value_counts()
values_count

Label
0.0    2025
Name: count, dtype: int64

In [15]:

from PIL import Image


def check_image(image_list:str):
    image_list = [f'raw_images/{image}' for image in image_list.split()]
    for image_file in image_list:
        if not os.path.exists(image_file):
            continue
        try:
            with Image.open(image_file) as img:
                img.verify()
            
            return image_file
        except (IOError, SyntaxError) as e:
            print(f"图片 {image_file} 无效: {e}")
            continue
    return None
    
df['QueryImages'] = df['QueryImages'].apply(lambda x: check_image(x))
df.dropna(subset=['QueryImages'], inplace=True)
len(df)

2025

In [16]:
df.to_csv('politifact.csv')